In [1]:
import tensorflow as tf    
from tensorflow import keras
import numpy as np

In [2]:
all_devices = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", all_devices)
physical_devices=tf.config.experimental.list_physical_devices('GPU')
gpus= tf.config.experimental.list_physical_devices('GPU')
for i in range(0,all_devices):
    tf.config.experimental.set_memory_growth(gpus[i], True)

mirrored_strategy = tf.distribute.MirroredStrategy(devices=[f"/GPU:{GPU_id}" for GPU_id in range (0,6)])

2021-12-09 21:15:19.248939: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-09 21:15:19.432944: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:3b:00.0
2021-12-09 21:15:19.434310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5e:00.0
2021-12-09 21:15:19.435626: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5f:00.0
2021-12-09 21:15:19.436883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:88:00.0
2021-12-09 21:15:19.438128: I tensorflow/core/common_ru

Num GPUs Available:  6


2021-12-09 21:15:20.284053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:3b:00.0
2021-12-09 21:15:20.284972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5e:00.0
2021-12-09 21:15:20.285845: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5f:00.0
2021-12-09 21:15:20.286771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:88:00.0
2021-12-09 21:15:20.287690: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 4 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:af:

In [3]:
X_train, X_test, y_train, y_test = np.load('full_data_binary.npy', allow_pickle=True)
X_train.shape

(327177, 11)

In [4]:
y_train, y_test = y_train.values, y_test.values

In [5]:
# Imbalanced dataset so want to adjusts weights of signal and background training examples
tot = len(y_train)
pos = np.sum(y_train == 1)
neg = tot - pos
print(f'Total training samples:  {tot} \npositives:  {pos} \nnegatives:  {neg}')

# weight positives more than negatives
weight_for_0 = (1 / neg) * (tot / 2.0)
weight_for_1 = (1 / pos) * (tot / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print(f'Postive weight:  {weight_for_1} \nNegative weight:  {weight_for_0}')

Total training samples:  327177 
positives:  79444 
negatives:  247733
Postive weight:  2.0591674638739237 
Negative weight:  0.660341981084474


In [6]:
import keras_tuner as kt

def build_model(hp):
    model = keras.models.Sequential([
                                   keras.layers.Flatten(input_shape=(11,))
  ])
    for _ in range(hp.Int('layers', 2, 5)):
        model.add(keras.layers.Dense(hp.Int('units', min_value=100, max_value=500, step=100)))
        if hp.Boolean("dropout"):
            model.add(keras.layers.Dropout(rate=0.2))
    
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=hp.Choice('optimizer', ['Adam', 'Nadam']), metrics=[keras.metrics.BinaryAccuracy(name='accuracy'), keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall')])
    return model

tuner = kt.RandomSearch(build_model, objective='val_loss', max_trials=20, overwrite=True)



2021-12-09 21:15:21.375118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:3b:00.0
2021-12-09 21:15:21.376712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5e:00.0
2021-12-09 21:15:21.378188: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:5f:00.0
2021-12-09 21:15:21.379646: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:88:00.0
2021-12-09 21:15:21.382158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 4 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:af:

In [7]:
tuner.search(X_train, y_train, epochs=10,  validation_data=(X_test,y_test), workers=all_devices, class_weight=class_weight)

Trial 20 Complete [00h 10m 53s]
val_loss: 0.5520776534183982

Best val_loss So Far: 0.5509150295270977
Total elapsed time: 03h 02m 19s
INFO:tensorflow:Oracle triggered exit


In [8]:
bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
bestHP.values

{'layers': 3, 'units': 400, 'dropout': False, 'optimizer': 'Adam'}

In [9]:
best_model = build_model(bestHP)
best_model.save('Event_best_model_1_epochs.h5')

In [10]:
# Set up tensorboard stuff
import os
import time


def get_run_logdir():
    """Create filename for log with date and time""" 
    root_logdir = os.path.join(os.curdir, 'my_logs')
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") 
    return os.path.join(root_logdir, run_id)

tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir())

In [11]:
history = best_model.fit(X_train, y_train, epochs=20, validation_data=[X_test, y_test], class_weight=class_weight, callbacks=[keras.callbacks.EarlyStopping(patience=5),tensorboard_cb])
best_model.save('Event_best_model_20_epochs.h5')

Train on 327177 samples, validate on 81795 samples
Epoch 1/20
    32/327177 [..............................] - ETA: 6:07:23 - loss: 0.8707 - accuracy: 0.4688 - precision: 0.3889 - recall: 0.5385

2021-12-10 00:17:44.780783: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
2021-12-10 00:17:45.400004: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcupti.so.10.0
2021-12-10 00:17:46.136924: I tensorflow/core/platform/default/device_tracer.cc:588] Collecting 122 kernel records, 19 memcpy records.


327177/327177 [==============================] - 48s 146us/sample - loss: 0.5556 - accuracy: 0.7240 - precision: 0.4566 - recall: 0.7171 - val_loss: 0.5559 - val_accuracy: 0.7298 - val_precision: 0.4631 - val_recall: 0.7088
Epoch 2/20
327177/327177 [==============================] - 43s 131us/sample - loss: 0.5532 - accuracy: 0.7255 - precision: 0.4584 - recall: 0.7187 - val_loss: 0.5575 - val_accuracy: 0.7193 - val_precision: 0.4517 - val_recall: 0.7304
Epoch 3/20
327177/327177 [==============================] - 46s 141us/sample - loss: 0.5530 - accuracy: 0.7254 - precision: 0.4583 - recall: 0.7195 - val_loss: 0.5527 - val_accuracy: 0.7340 - val_precision: 0.4682 - val_recall: 0.7032
Epoch 4/20
327177/327177 [==============================] - 44s 133us/sample - loss: 0.5527 - accuracy: 0.7253 - precision: 0.4582 - recall: 0.7182 - val_loss: 0.5514 - val_accuracy: 0.7267 - val_precision: 0.4597 - val_recall: 0.7167
Epoch 5/20
327177/327177 [==============================] - 43s 132us/s

In [ ]:
from tensorflow import keras

tuned_model = keras.models.load_model("Event_best_model_20_epochs.h5")
# Get predictions for test set
y_pred = tuned_model.predict()

In [12]:
# class Model:

#     def __init__(self, data_path='SL_Higgs_input_data.npy', model):

#         data = np.load(data_path)
#         self.X_train, self.X_test, self.y_train, self.y_test = data
#         self.model = model
#         tot = len(self.y_train)
#         pos = np.sum(self.y_train == 1)
#         neg = tot - pos
#         weight_for_0 = (1 / neg) * (tot / 2.0)
#         weight_for_1 = (1 / pos) * (tot / 2.0)
#         self.weights = {0: weight_for_0, 1: weight_for_1}
        
    
#     def fit(self, epochs=50):
#         self.history = self.model.fit()

        




In [13]:
# Compile model with PR metrics as well as accuracy 
# model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=[keras.metrics.BinaryAccuracy(name='accuracy'),
#       keras.metrics.Precision(name='precision'),
#       keras.metrics.Recall(name='recall')])

In [14]:
# # Set up tensorboard stuff
# import os
# import time


# def get_run_logdir():
#     """Create filename for log with date and time""" 
#     root_logdir = os.path.join(os.curdir, 'my_logs')
#     run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") 
#     return os.path.join(root_logdir, run_id)

# tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir())

In [15]:
# fit model for 20 epochs

# history = model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test), callbacks=[tensorboard_cb], class_weight=class_weight)

In [16]:
# from sklearn.metrics import roc_curve

# y_pred = model.predict(X_test_scaled).ravel()
# fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [17]:
# import matplotlib.pyplot as plt

# plt.plot(fpr, tpr)
# plt.plot(np.linspace(0,1,100), np.linspace(0,1,100), linestyle='dashed', linewidth=0.5, color='k')
# plt.xlabel('fpr')
# plt.ylabel('tpr')
# plt.savefig('ROC_basic_model.png', dpi=300)
# plt.show()

In [18]:
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import RandomizedSearchCV

# def build_model(units, layers):
#   model = keras.models.Sequential([
#                                    keras.layers.Flatten(input_shape=(11,))
#   ])
#   for _ in range(layers):
#     model.add(keras.layers.Dense(units, activation='selu', kernel_initializer='lecun_normal'))
#   model.add(keras.layers.Dense(1, activation='sigmoid'))
#   model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=[keras.metrics.BinaryAccuracy(name='accuracy'),
#       keras.metrics.Precision(name='precision'),
#       keras.metrics.Recall(name='recall')])
#   return model


# model = KerasClassifier(build_fn=build_model)
# param_distribs = {'units':[50,100,200,300], 'layers':[1, 2, 3, 4, 5]}
# rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=2, cv=2)

In [19]:
# rnd_search_cv.fit(X_train, y_train, epochs=2, validation_data=[X_test, y_test], class_weight=class_weight, callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [20]:
# best_params = rnd_search_cv.best_params_
# model = rnd_search_cv.best_estimator_.model
# model.save('cv_event_level_best_model')

In [21]:
# model = keras.models.Sequential(([
#     keras.layers.Flatten(input_shape=(11,)),
#     keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
#     keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
#     keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
#     keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
#     keras.layers.Dense(1, activation='sigmoid')
# ]))

# model.summary()

In [22]:
# # Compile model with PR metrics as well as accuracy 
# model.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=[keras.metrics.BinaryAccuracy(name='accuracy'),
#       keras.metrics.Precision(name='precision'),
      # keras.metrics.Recall(name='recall')])

In [23]:
#  Save best model
# checkpoint_cb = keras.callbacks.ModelCheckpoint("second_model.h5", save_best_only=True)

# history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), class_weight=class_weight, batch_size=32)

In [24]:
# complex_model = keras.models.load_model('second_model.h5')
# complex_model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test), callbacks=[tensorboard_cb, checkpoint_cb], class_weight=class_weight)